In [1]:
# 初始化
import django_setup

## tiptap / docx转化测试

In [2]:
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_doc_chap3 - 1pages_无嵌套 .docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp1_short.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_doc_chap3 - 1pages_有嵌套.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/caseTest/Case6_中国区零食包框架供应商采购项目招采文件.docx"
doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/实战：招标文件-第1包：一级压榨花生油 - 副本.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp3.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp3 p1.docx"

# 转换为WSL路径
doc_path_wsl  = doc_path_win.replace("C:", "/mnt/c").replace("\\", "/")
print(f"WSL路径: {doc_path_wsl}")


import os
print(f"文件是否存在: {os.path.exists(doc_path_wsl)}")

WSL路径: /mnt/c/Users/huiwa/Downloads/文本分析测试/test_files/实战：招标文件-第1包：一级压榨花生油 - 副本.docx
文件是否存在: True


In [3]:
from apps.projects.tiptap.docx import docx_to_html
html_content = docx_to_html(doc_path_wsl)
from pprint import pprint
pprint(html_content)

WARNING 2025-04-25 07:43:33,293 docx DOCX转换警告: Message(type='warning', message='Did not understand this style mapping, so ignored it: /* 添加警告中提到的样式 */')
WARNING 2025-04-25 07:43:33,293 docx DOCX转换警告: Message(type='warning', message="Did not understand this style mapping, so ignored it: p[style-id='2'] => p.body-text")
WARNING 2025-04-25 07:43:33,294 docx DOCX转换警告: Message(type='warning', message="Did not understand this style mapping, so ignored it: p[style-id='6'] => p.table-text")


('<p></p><p></p><p></p><p></p><p></p><p '
 'class="body-text"><strong>北京京铁运恒采购供应站有限公司</strong> <strong>2024</strong> '
 '<strong>年端午节物资采购项目</strong></p><p></p><p></p><p></p><p></p><p></p><p></p><p></p><p '
 'class="body-text"><strong>招标文件</strong></p><p></p><p></p><p></p><p></p><p></p><p></p><p></p><p '
 'class="body-text"><strong>项目名称</strong>：<strong>北京京铁运恒采购供应站有限公司</strong> '
 '<strong>2024</strong> <strong>年端午节</strong> '
 '<strong>物资采购项目（第一包：一级压榨花生油）</strong><strong>项目编号：DLXM-2024-148-01</strong><strong>招</strong> '
 '<strong>标</strong> <strong>人：北京京铁运恒采购供应站有限公司</strong> '
 '<strong>代理机构：北京中外建工程管理有限公司</strong></p><p></p><p></p><p></p><p></p><p></p><p></p><p></p><p '
 'class="body-text"><strong>2024</strong> <strong>年</strong> '
 '<strong>3</strong> <strong>月</strong></p><p></p><p '
 'class="body-text"><strong>目</strong>    '
 '<strong>录</strong></p><p></p><p></p><p class="body-text"><a '
 'id="bookmark1"></a><strong><a href="#bookmark2">第一章</a></strong><a '
 'href="#bookmark2">  <

In [4]:
from IPython.display import display, HTML
display(HTML(html_content))

In [ ]:
from apps.projects.tiptap.docx import docx_to_tiptap_json
json_content = docx_to_tiptap_json(doc_path_wsl)
from pprint import pprint
pprint(json_content)

## 集成测试

In [2]:
from apps.projects.models import Project, TaskStatus

In [ ]:
from apps.projects.models import Task, TaskType
task = Task.objects.get(stage__project=current_project, type=TaskType.DOCX_EXTRACTION_TASK)
print(task.docx_tiptap)

In [ ]:
current_project = Project.objects.get(project_name='2')
print(f"用于测试的项目: {current_project.project_name}")
print(f"项目包含的文件: {current_project.files.all()}")


In [ ]:
task.docx_tiptap = "1"
task.save()
print(task.docx_tiptap)

In [ ]:
from django.core.exceptions import ValidationError
import os, tempfile, requests, uuid
import logging
logger = logging.getLogger(__name__)

# 获取文件的预签名URL
presigned_url = current_project.files.first().get_presigned_url()
if not presigned_url:
    raise ValidationError("无法获取文件访问地址")

logger.info(f"开始下载文件: project_id={current_project.id}, file={current_project.files.first().name}")

# 下载文件到临时文件
temp_file_path = os.path.join(tempfile.gettempdir(), f"doc_analysis_{uuid.uuid4()}.docx")
response = requests.get(presigned_url)
response.raise_for_status()

with open(temp_file_path, 'wb') as temp_file:
    temp_file.write(response.content)

In [ ]:
task.docx_tiptap = json_content
task.save()
print(task.docx_tiptap)